# XNOR-Net
[XNOR-Net: ImageNet Classification Using Binary Convolutional Neural Networks](https://arxiv.org/pdf/1603.05279.pdf)

## Related Work

### Shallow networks

### Compressing pre-trained deep networks

主要思想是减少冗余的参数，有些神经元的参数比较小，作用也比较小，所以可以去掉。

- Weight decay [cite 17]
- the Hessian of loss function: Optimal Brain Damage [cite 18] and Optimal Brain Surgeon [cite 19]
- ...

### Designing compact layers

研究网络结构，减少计算量。

Using in Network in Network architecture, GoogLenet and Residual-Net

In Residual-Net, bottleneck structure has benn proposed.

### Quantizing parameters

量化参数

### Network binarization

- Expectation BackPropagation(EBP) [cite 36]
- Bayesian approach
- BinaryConnect [cite 38]
- BinaryNet [cite 11]

# Binary Convolutional Neural Network

$L$-layer CNN architecture: $ <\mathcal{I}, \mathcal{W}, * > $

$I = \mathcal{I} _ {l(l=1,\dots,L)}$ is the input tensor for the $l^{\mathrm{th}}$ layer of CNN.

$W = \mathcal{W} _ {k=1,\dots, K^l}$ is the $k^{\mathrm{th}}$ weight filter in the $l^{\mathrm{th}}$ layer of the CNN.

$K^l$ is the number of weight filters in the $l^{\mathrm{th}}$ layer of CNN.

$*$ represents a convolutional operation with $I$ and $W$ as it operands(操作数).

$I \in \mathbb{R}^{c \times w_{in} \times h_{in}}$, where $(c, w_{in}, h_{in})$ represents *channels, width* and *height* respectively.

$W \in \mathbb{R} ^ {c \times w \times h}$, where $w \leq w_{in}, h \leq h_{in}$.

## Binary-Weight-Networks

**Result:**
- is smaller ($\sim 32 \times$) than an equivalent network with single-precision weights values.
- in $\sim 2\times$ speed up.

**Goal:**
$$W \approx \alpha B,\;\;\;\;\alpha \in \mathbb{R}^{+}, B \in \{+1, -1\}^{c \times w \times h}$$

So, $$I * W \approx (I \oplus B) \alpha$$
where $\oplus$ indicates a convolution without any multiplication.

** optimization **
$$B^{*} = \mathrm{sign}(W)$$
$$\alpha^{*} = \frac{1}{n} \| W \| _{\ell 1} = \frac{\sum{|W|}}{n} = \frac{\sum{B \odot W}}{n}$$

** Train **

$$\frac{\partial \mathrm{sign}}{\partial r} = r1_{|r| \leq 1}$$

$$\frac{\partial C}{\partial W} = \frac{\partial C}{\tilde{W}}(\frac{1}{n} + \frac{\partial \mathrm{sign}}{\partial W} \alpha)$$

let $$C = I*W \approx I * (B \alpha) = (I \oplus B) \alpha = M \alpha, M = I \oplus B$$
so, $$\frac{\partial C}{\partial \alpha} = \sum{(M \odot C')}$$
$$\frac{\partial \alpha}{\partial W} = B \odot \frac{1}{n}$$
$$\frac{\partial C}{\partial M} = \alpha C'$$
$$\frac{\partial M}{\partial I} = (M * B)^{\mathrm{flip}} \odot M'$$
$$\frac{\partial M}{\partial B} = I * M'$$
$$\frac{\partial B}{\partial W} = \frac{\partial \mathrm{sign}}{\partial W}$$
$$\frac{\partial C}{\partial W} = \frac{\partial C}{\partial \alpha} \frac{\partial \alpha}{\partial W} + \frac{\partial C}{\partial M}\frac{\partial M}{\partial B} $$

## XNOR-Networks

**Result:**
- $\sim 58 \times$ speed up.

改进思路

1. 输入层改进：
    - （原）实数输入，Batch Normalization之后直接输入
    - 增广通道数，原图片每一像素的每一bit都作为一个通道，即3通道变为24通道
    - 增加一实数卷积层，已增加通道数，保留更多的信息。
    
2. 卷积层改进
    - 参数的二值化变为三值化，即{-1,0,1}，用多一倍的计算时间和空间换取精度。
    - 尝试其他梯度反向传播函数
    - padding对二值化的影响？

3. 梯度二值化
    - 改变反向传播方法